In [ ]:
import utils.data as ud
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

In [ ]:
import tbtools.dev as tbdev
import tbtools.panda as tbpd

## Assign train, validate, test labels

In [ ]:
from sklearn.cross_validation import train_test_split

import utils.features as uf

In [ ]:
%%time
x,y = uf.get_x_y( c_lags=('10 min',
                          '20 min',
                          '30 min'),
                  dc_settings=(
                    ('10 min', '2 min'),
                    ('20 min', '2 min'),
                    ('30 min', '2 min'),
                    ('10 min', '5 min'),
                    ('20 min', '5 min'),
                    ('30 min', '5 min'),
                ))

In [ ]:
fig = sns.plt.figure()
sns.plt.plot((x.index - x.index.min()).total_seconds()[::50])

In [ ]:
x['C'] = y

_train, _test = train_test_split(x, test_size=0.2, random_state=0)
_train, _validate = train_test_split(_train, test_size=.25, random_state=1)

In [ ]:
def xy(v):
    return v.drop('C', axis=1), v['C']

x_train, y_train = xy(_train)
x_validate, y_validate = xy(_validate)
x_test, y_test = xy(_test)

In [ ]:
x_train.shape, x_validate.shape, x_test.shape

---

# Plot variables against target

In [ ]:
tbpd.hist2d(x_train.iloc[:,0], y_train, integer_aligned_bins=True, square=True);
sns.plt.figure()
tbpd.hist2d(x_train.iloc[:,1], y_train, integer_aligned_bins=True, square=True);
sns.plt.figure()
tbpd.hist2d(x_train.iloc[:,2], y_train, integer_aligned_bins=True, square=True);


---

In [ ]:
perfs = pd.DataFrame()

In [ ]:
perfs.sort_values('rmse')

## Model validation

In [ ]:
import sklearn.metrics as skmet

def score(model):
    pred = model.predict(x_validate)
    
    if isinstance(model, Pipeline):
        model = model.steps[-1][1]
    name = str(model)
    
    d = {
        'rmse': np.sqrt(skmet.mean_squared_error(y_validate, pred)),
        'mae': skmet.mean_absolute_error(y_validate, pred),
        'evs': skmet.explained_variance_score(y_validate, pred),
        'r2': skmet.r2_score(y_validate, pred),
    }
    s = pd.Series(d, name=name)
    
    global perfs
    perfs.append(s)
    
    return s

In [ ]:
def plot_residuals(model):
    res = y_train - model.predict(x_train)
    n = x_train.shape[1]
    for i in range(n):
        sns.plt.figure(figsize=(6,5))
        x = x_train.iloc[:,i]
        tbpd.hist2d(x, res, 
                    bins=((max(x) - min(x)), 20),
                    vlabel=x_train.columns[i],
                    hlabel='Residuals',
                   )

In [ ]:
import tbtools.dev as tbdev
import tbtools.panda as tbpd
tbpd = tbdev.reload(tbpd)

Consider QQ plot

In [ ]:
plot_residuals(p)

In [ ]:
from sklearn.preprocessing import 

In [ ]:
MinMaxScaler??

In [ ]:
class FeaturePicker:
    def __init__(self, keep):
        self.keep = keep
        
    def transform(self, x):
        return x[self.keep]
    
    def fit(self, *args):
        # do nothing
        return self
    

# Models

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

## Linear regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()
mms = MinMaxScaler()

In [ ]:
p = Pipeline( (('MinMaxScaler', mms), ('LinearRegression', lr)) )
p.fit(x_train, y_train)
score(p)

In [ ]:
name = x_train.columns[0]
a = x_train[name].resample('H').mean()
b = y_train.resample('H').mean()

sns.plt.figure(figsize=(15,8))
sns.plt.scatter(a, b)
sns.plt.ylabel('y')

In [ ]:
name = x_train.columns[1]
a = x_train[name].resample('H').mean()
b = y_train.resample('H').mean()

sns.plt.figure(figsize=(15,8))
sns.plt.scatter(a, b)
sns.plt.ylabel('y')
sns.plt.xlabel(name)

In [ ]:
name = x_train.columns[2]
a = x_train[name].resample('H').mean()
b = y_train.resample('H').mean()

sns.plt.figure(figsize=(15,8))
sns.plt.scatter(a, b)
sns.plt.ylabel('y')
sns.plt.xlabel(name)

In [ ]:
sns.plt.figure(figsize=(15,8))
y_train.resample('H').mean().plot(linewidth=0.5)
sns.plt.ylabel('Reinspection count')

In [ ]:
sns.plt.figure(figsize=(15,8))
name = x_train.columns[0]
x_train[name].resample('H').mean().plot(linewidth=0.5)
sns.plt.ylabel(name)

In [ ]:
p = Pipeline((
        ('Only 5 min', FeaturePicker(['ΔC L=5 min W=2 min', 'C L=5 min'])), 
        ('MinMaxScaler', mms), 
        ('LinearRegression', lr)) )
p.fit(x_train, y_train)
score(p)

## Elastic net

In [ ]:
from sklearn.linear_model import ElasticNetCV

In [ ]:
en = ElasticNetCV(l1_ratio=(0.1, 0.3, 0.5, 0.7, 0.8, 0.9, 0.95, 0.99))
mms = MinMaxScaler()
p = Pipeline( (('MinMaxScaler', mms), ('ElasticNetCV', en)) )
p.fit(x_train, y_train)
score(p)

## Decision tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
DecisionTreeRegressor?

In [ ]:
p = Pipeline((
#         ('Only 5 min', FeaturePicker(['ΔC L=5 min W=2 min', 'C L=5 min'])), 
#         ('MinMaxScaler', mms), 
        ('DT', DecisionTreeRegressor()),
    ))
p.fit(x_train, y_train)
score(p)

## RF

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
p = Pipeline((
#         ('Only 5 min', FeaturePicker(['ΔC L=5 min W=2 min', 'C L=5 min'])), 
#         ('MinMaxScaler', mms), 
        ('RF', RandomForestRegressor()),
    ))
p.fit(x_train, y_train)
score(p)

In [ ]:
x_train.columns

Sure looks like the most important feature is C 5 minutes ago...

In [ ]:
en.coef_

In [ ]:
rf = p.steps[-1][1]
rf.feature_importances_

---

## Straight analysis stuff

The following figure depicts the autocorrelation for C, excluding nights and weekends. It is not impressive.

In [ ]:
sns.plt.figure(figsize=(15,8))
res = c.resample('H').mean().fillna(0)
res = res[res.index.dayofweek < 5]
res = res[res.index.hour > 5]
res = res[res.index.hour < 20]
pd.tools.plotting.autocorrelation_plot(
# c[(c.index.hour>6) & (c.index.hour<20) & (c.index.dayofweek < 5)].resample('B').mean().fillna(0).values
    res.values
)
sns.plt.vlines(np.arange(0,sns.plt.xlim()[1], 24), -1, 1, color='w', alpha=0.75)
sns.plt.vlines(np.arange(0,sns.plt.xlim()[1], 5*24), -1, 1, color='k', alpha=0.75)
